<a href="https://colab.research.google.com/github/Suryageeks/Smart-Car-Parking-Detection-System/blob/main/Training/Parking_Frontend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 61.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.5.0+cu101 which is incompatible.
     |████████████████████████████████| 274 kB 27.3 MB/s 
  Created wheel f

In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
     |████████████████████████████████| 6.2 MB 642 kB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60944 sha256=e5339c99b3f2a8f6050f84a22154a9c54173c0bf574a5f28891b871a28da5ccb
  Stored in directory: /root/.cache/pip/wheels/16/98/fc/252d62cab6263c719120e06b28f3378af59b52ce7a20e81852
Successfully built fvcore


In [ ]:
!pip install streamlit

     |████████████████████████████████| 9.3 MB 18.1 MB/s 
     |████████████████████████████████| 4.3 MB 43.8 MB/s 
     |████████████████████████████████| 164 kB 63.0 MB/s 
     |████████████████████████████████| 180 kB 66.7 MB/s 
     |████████████████████████████████| 111 kB 56.0 MB/s 
     |████████████████████████████████| 76 kB 220 kB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 127 kB 64.0 MB/s 
     |████████████████████████████████| 792 kB 53.4 MB/s 
     |████████████████████████████████| 374 kB 61.6 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=79d3c734796c09f3af5116c7d6b6a05ceec71806ac9a5a3c338f9e1dca0e5f40
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 26.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=12cbb5f5cd26a6b393e9d53935284169ef528f25312a4187e08493b5f5684178
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!pip install plotly

In [ ]:
pip install streamlit-folium

     |████████████████████████████████| 95 kB 3.8 MB/s 
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.12.1.post1 which is incompatible.


In [ ]:
import numpy

In [ ]:
pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [ ]:
%%writefile app.py
import streamlit as st
from detectron2.utils.logger import setup_logger
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from PIL import Image
import pandas as pd
import requests
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from streamlit_folium import folium_static
import folium

setup_logger()

title = '<p style="font-family:monospace; color:orange; font-size: 40px;"><b>Smart Car Parking Detection System</b></p>'
st.markdown(title,unsafe_allow_html=True)

st.sidebar.subheader('**Upload a File**')
file_upload = st.sidebar.file_uploader("Choose a Image",type=['png','jpeg','jpg'])


if file_upload is not None:
  #get file details
  file_details = {"filename":file_upload.name, "filetype":file_upload.type,"filesize":file_upload.size}
  st.sidebar.markdown('**File Details**')
  st.sidebar.write(file_details)

  st.subheader('**Input Image**')  
  file_bytes = np.asarray(bytearray(file_upload.read()), dtype=np.uint8)  
  opencv_image = cv2.imdecode(file_bytes, 1)
  st.image(opencv_image, channels="BGR",width=380)

st.write('\n')

cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"
cfg.merge_from_file('/content/drive/MyDrive/Parking Inference/config.yml')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3  
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/Parking Inference/model_final.pth'
predictor = DefaultPredictor(cfg)
outputs = predictor(opencv_image)

c,k=0,0
for i in outputs["instances"].pred_classes:
  if i==1:
    c=c+1
  elif i==2:
    k=k+1

st.subheader('**Inferenced Image**')
v = Visualizer(opencv_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
st.image(out.get_image()[:, :, ::-1],caption='Processed Image',width=380)
st.write('**Inferenced Details**')
st.markdown(f"Available Parking Space: {c}")
st.markdown(f"Filled Parking Space: {k}")

# Visualize
st.subheader('**Visualize**')
d = {'X_axis': ['Available Parking Space','Filled Parking Space'],
     'Y_axis': [c,k]}

df = pd.DataFrame(d)
fig = px.bar(        
        df,
        x = "X_axis",
        y = "Y_axis",
        title = "Stats",
        color="X_axis"        
    )
st.plotly_chart(fig)

st.markdown('**Parking Location Map**')
m = folium.Map(location=[22.5629,88.3962],width=550, height=352,zoom_start=80,zoom_control=False)
folium.Marker(
    location=[22.5629,88.3967],
    popup="Parking Available {}/{}".format(c,(c+k)),
    icon=folium.Icon(prefix="fa",color="orange",icon="car"),
    tiles='CartoDB Positron'
).add_to(m)

folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)

folium_static(m)

#Message API
#API KEY
headers={
    "authorization":"DqMpvl7mny8YH2KcWJBQCuo6gS035G1hXbdk9iTPLeFaAfNzxs5I0QOZxBpGJg9PRXEWCjfUkKH6VcTl",    
    "Content-Type":"application/x-www-form-urlencoded"
}

st.subheader('**Parking Kiosk Information**')
numbers= st.text_input('Enter Mobile Number')
url='https://www.fast2sms.com/dev/bulkV2'
message= f'Parking Space Available - {c}'
payload= f'sender_id=TXTIND&message={message}&route=v3&language=english&numbers={numbers}'
if st.button('Send'):
  response=requests.request('POST',url=url,data=payload,headers=headers)
  st.success('**Message Sent Successfully**')


Overwriting app.py


In [ ]:
!ngrok authtoken 1sxmRs5h5d7BDAp7YxTwILbkH9N_6meKLhsKjFZgRx7trFwmW

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok

In [ ]:
!streamlit run --server.port 80 app.py&>/dev/null&

In [ ]:
!pgrep streamlit

512
982


In [ ]:
from pyngrok import ngrok
url = ngrok.connect()
ssh_tunnel = ngrok.connect(22, "tcp")

In [ ]:
url

<NgrokTunnel: "http://8dcf-34-86-25-166.ngrok.io" -> "http://localhost:80">

In [ ]:
ngrok.kill()